In [1]:
import os
import pandas as pd

from datetime import datetime
from utils.config import DRLConfig
from utils.drl_train import training_pipeline

%load_ext autoreload
%autoreload 2

In [2]:
df_ret = pd.read_parquet("../data/returns.parquet")
df_prices = pd.read_parquet("../data/prices.parquet")
df_vol = pd.read_parquet("../data/vola.parquet")

To view the logs:
1. Open a terminal or command prompt.
2. Navigate to the directory *containing* the `logs` directory (i.e., the root of this repository).
3. Run the command: `tensorboard --logdir logs/`
4. Open the URL provided by TensorBoard (usually http://localhost:6006/) in your web browser.

You should see experiments named like `PPO_WindowX_AgentY_SeedZ`.

In [3]:
# Create timestamp for this run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create configuration
config = DRLConfig(

    # Window configuration
    n_windows=1,  # 10 in paper
    agents_per_window=5,  # 5 in paper
    base_start_year=2006,

    # Environment parameters
    env_window_size=60,
    transaction_cost=0.0,
    initial_balance=100_000,
    reward_scaling=1.0,
    eta_dsr=1 / 252,

    # Training parameters
    n_envs=10,
    total_timesteps_per_round= 1000, # 7_500_000, # 7_500_000 in paper
    n_steps_per_env=252 * 3,
    batch_size=1260,
    n_epochs=16,
    gamma=0.9,
    gae_lambda=0.9,
    clip_range=0.25,
    log_std_init=-1.0,

    # Learning rate parameters
    initial_lr=3e-4,
    final_lr=1e-5,

    # Paths
    model_save_dir=f"../models/{timestamp}",
    tensorboard_log_dir=f"../logs/{timestamp}",
)

In [4]:
# Run training pipeline
results, backtest_portfolio = training_pipeline(
    drl_config = config,
    df_prices = df_prices,
    df_ret = df_ret,
    df_vol = df_vol
)

# Convert results to DataFrame
results_df = pd.DataFrame(results)

--- Starting Window 1/1 (Train Year Start: 2006) ---
  Train Period: 2006-01-01 to 2010-12-31
  Val Period  : 2011-01-01 to 2011-12-31
  Test Period : 2012-01-01 to 2012-12-31
  Training Agent 1/5 with seed 0...


Output()

    Starting training for 1000 timesteps...



Training complete. Trained for 1000 timesteps.
TensorBoard logs for experiment 'PPO_Seed0' saved in directory: ../logs/20250602_222332
    Evaluating agent on validation set...
    Validation Mean Reward: -2.2366
    Agent saved to: ../models/20250602_222332/agent_seed0_valrew-2.24.zip
  Training Agent 2/5 with seed 1...


Output()

...

    Starting training for 1000 timesteps...



Training complete. Trained for 1000 timesteps.
TensorBoard logs for experiment 'PPO_Seed3' saved in directory: ../logs/20250602_222332
    Evaluating agent on validation set...
    Validation Mean Reward: -3.1748
    Agent saved to: ../models/20250602_222332/agent_seed3_valrew-3.17.zip
  Training Agent 5/5 with seed 4...


Output()

    Starting training for 1000 timesteps...



Training complete. Trained for 1000 timesteps.
TensorBoard logs for experiment 'PPO_Seed4' saved in directory: ../logs/20250602_222332
    Evaluating agent on validation set...
    Validation Mean Reward: -1.6149
    Agent saved to: ../models/20250602_222332/agent_seed4_valrew-1.61.zip
    Running backtest evaluation...


In [5]:
results_filename = f"backtest_results_summary_{timestamp}.csv"
results_save_path = os.path.join(config.model_save_dir, results_filename)
results_df.to_csv(results_save_path, index=False)
print(f"\nBacktest results summary saved to: {results_save_path}")
print("\nFinal Results DataFrame:")
results_df.head()


Backtest results summary saved to: ../models/20250602_222332/backtest_results_summary_20250602_222332.csv

Final Results DataFrame:


,window,best_agent_path,status,metrics
0,1,../models/20250602_222332/agent_seed4_valrew-1...,completed,"{'n_eval_episodes': 1, 'mean_reward': 11.98083..."
